In [6]:
%matplotlib widget
import warnings
warnings.filterwarnings("ignore", module="torch.nn.functional")

from pathlib import Path

from image_helpers import show_im, show_im_points
from smallUnet import *
from skyrm_find_CNN import *

if torch.cuda.is_available():
    gpu = 0
    print(f"There are {torch.cuda.device_count()} GPUs available")
    print(f"Running on GPU (index) {gpu}")
else:
    gpu = "cpu"
    print("GPU is not available, will run on CPU.")

savedir = Path("./NN_trained/").absolute()


There are 4 GPUs available
Running on GPU (index) 0


#### If loading a NN

In [3]:
nn_path = savedir / "SkX_NN_final.pt"
model = smallUnet()
model.cuda(gpu)
model.load_state_dict(torch.load(nn_path))
model.eval()

# specify optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)  ### need to do this? 

# Test a NN on a full image
Skip to here if already have a trained NN 
For actual experimental data, I have a workflow for bringing in the raw TEM images, filtering them, determining the tilt axis and compensating for tilt angle, etc., but for this example just using an already processed image. 

In [4]:
test_im = np.load("./test_data/test_image_600G_150K.npy")
tilt_dir = 132.137 # tilt axis
show_im(test_im, title="Example image")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
model = trained_NN(nn_path, cuda=True)
centers = model.find_skyrms(test_im, tilt_dir, thresh=0.3, gpu=gpu)
show_im(model.prediction[:,:,0], "Model prediction", simple=True)
show_im_points(test_im, centers, title="Skyrmion locations")

/home/amccray/anaconda3/envs/torch/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448255797/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …